In [ ]:
"""
This file handles data cleaning, preparation, and visualization
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def data_quality_assessment(df):
    """Perform a comprehensive data quality assessment"""
    quality_report = {
        'total_records': len(df),
        'missing_values': {},
        'duplicates': {},
        'data_types': {}
    }
    
    # Missing Values
    missing_values = df.isnull().sum()
    quality_report['missing_values'] = missing_values[missing_values > 0].to_dict()
    
    # Duplicates
    duplicates = df[df.duplicated()]
    quality_report['duplicates'] = {
        'total_duplicates': len(duplicates),
        'sample_duplicates': duplicates.to_dict('records')[:5]
    }
    
    # Data Type Inconsistencies
    for column in df.columns:
        try:
            pd.to_numeric(df[column], errors='raise')
        except ValueError:
            quality_report['data_types'][column] = df[column].dtype
    
    return quality_report

def handle_missing_values(df):
    """Handle missing values by filling or imputing"""
    if 'UNIT PRICE' in df.columns:
        df['UNIT PRICE'] = df['UNIT PRICE'].str.replace(',', '').astype(str)
        df['UNIT PRICE'] = pd.to_numeric(df['UNIT PRICE'], errors='coerce')
        median_price = df['UNIT PRICE'].median()
        df['UNIT PRICE'].fillna(median_price, inplace=True)
    return df

def handle_duplicates(df):
    """Remove duplicate rows"""
    return df.drop_duplicates()

def feature_engineering(df):
    """Perform feature engineering tasks"""
    df_engineered = df.copy()
    df_engineered['DATE'] = pd.to_datetime(
        df_engineered['DATE'], format='%B %d, %Y, %I:%M %p', errors='coerce'
    )
    df_engineered = df_engineered.dropna(subset=['DATE'])
    df_engineered['Month-Year'] = df_engineered['DATE'].dt.strftime('%B %Y')
    df_engineered['Total_Value'] = df_engineered['QUANTITY'] * df_engineered['UNIT PRICE']
    df_engineered['Day_of_Week'] = df_engineered['DATE'].dt.day_name()
    df_engineered['Hour_of_Day'] = df_engineered['DATE'].dt.hour
    return df_engineered

def save_cleaned_data(df, file_name='cleaned_data.csv'):
    """Save cleaned dataset"""
    df.to_csv(file_name, index=False)
    print(f"Cleaned data saved to {file_name}")

def main_data_preparation(df):
    """Main function to orchestrate cleaning, feature engineering, and visualization"""
    # Data Quality Assessment
    quality_report = data_quality_assessment(df)
    
    # Handle Missing Values
    df = handle_missing_values(df)
    
    # Handle Duplicate Records
    df = handle_duplicates(df)
    
    # Perform Feature Engineering
    df_processed = feature_engineering(df)
    
    # Save Cleaned Data
    save_cleaned_data(df_processed)
    
    return df_processed, quality_report

# Usage
data = pd.read_csv('data.csv')
df_processed, quality_report = main_data_preparation(data)

print(df_processed[['DATE', 'Month-Year']].head())
print("\n Data Quality Report:")
print(quality_report)

Cleaned data saved to cleaned_data.csv
                 DATE   Month-Year
0 2024-08-18 21:32:00  August 2024
1 2024-08-18 21:32:00  August 2024
2 2024-08-18 21:32:00  August 2024
3 2024-08-18 21:32:00  August 2024
4 2024-08-18 21:32:00  August 2024

 Data Quality Report:
{'total_records': 333405, 'missing_values': {'UNIT PRICE': 8}, 'duplicates': {'total_duplicates': 3524, 'sample_duplicates': [{'DATE': 'January 6, 2024, 11:52 AM', 'ANONYMIZED CATEGORY': 'Category-91', 'ANONYMIZED PRODUCT': 'Product-1b48', 'ANONYMIZED BUSINESS': 'Business-20fc', 'ANONYMIZED LOCATION': 'Location-b125', 'QUANTITY': 1, 'UNIT PRICE': '3,680'}, {'DATE': 'July 9, 2024, 2:26 PM', 'ANONYMIZED CATEGORY': 'Category-104', 'ANONYMIZED PRODUCT': 'Product-af50', 'ANONYMIZED BUSINESS': 'Business-476c', 'ANONYMIZED LOCATION': 'Location-b27b', 'QUANTITY': 1, 'UNIT PRICE': '1,310'}, {'DATE': 'July 9, 2024, 2:26 PM', 'ANONYMIZED CATEGORY': 'Category-92', 'ANONYMIZED PRODUCT': 'Product-d09a', 'ANONYMIZED BUSINESS': 'Busin